In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# import logging
# import constants
# from pathlib import Path

# from google.auth.transport.requests import Request
# from google.oauth2.credentials import Credentials
# from google_auth_oauthlib.flow import InstalledAppFlow
# from googleapiclient.discovery import build
# from googleapiclient.errors import HttpError

In [ ]:
from pathlib import Path
from json import JSONDecodeError
import logging
from time import time
from os import path

import constants_GDrive
import constants
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth import exceptions as google_exceptions
from ratelimit import limits, sleep_and_retry


In [ ]:
def do_exit(msg=None, code=0):
    '''provide exit message and exit
    
    Args:
    '''
    status = ''
    if code > 0:
        status = f'Error {code}: '
    if msg:
        print(f'{status}{msg}')
    exit(code)

In [ ]:
logging.basicConfig(level=logging.DEBUG)

logger = logging.getLogger(__name__)
logger.setLevel('DEBUG')
logger.debug('foo')

In [ ]:
class GDriveError(Exception):
    pass

In [ ]:
class GDrive():
    def __repr__(self):
        return 'GDrive()'
    
    def __str__(self):
        return f'GDrive()'
    
    def __init__(self, secrets, scopes, cache='./', token='./token.json'):
        '''create a google drive interface for searching and returning file/folder information
        
        Args:
            secrets(Path): secrets json file obtained from https://console.cloud.google.com/cloud-resource-manager
            token(Path): file to cache auth information (typically within cache path)
        '''
        self.secrets = secrets
        self.scopes = scopes        
        self.token = token
        self.credentials = self.create_token(secrets=self.secrets, 
                                            scopes=self.scopes, 
                                            token=self.token)
        self.service = self.build_service(self.credentials)
        self.MIMETYPES = constants_GDrive.MIMETYPES
        self.CORPORA = constants_GDrive.CORPORA
        self.FILE_FIELDS = constants_GDrive.FILE_FIELDS
        self.FIELDS_DEFAULT = constants_GDrive.FIELDS_DEFAULT
        self.PAGESIZE = constants_GDrive.PAGESIZE
        
                
    
    
    @staticmethod
    def create_token(secrets, scopes, token):
        '''create credentials object
        
        Args:
            secrets(Path): path to secrets file
            scopes(list): list of scopes to use
            token(Path): path to store cached auth token
            
        Returns: credentials object
        '''
        token = Path(token)
        write_token = False
        age_token = time() - path.getatime(token)
        if token.exists() and age_token < constants_GDrive.TOKEN_MAX_AGE:
            try:
                logger.debug(f'creating credentials token: {token} with scopes: {scopes}')
                creds = Credentials.from_authorized_user_file(token, scopes)
            except (OSError, JSONDecodeError) as e:
                logger.warning(f'rewriting token file due to error: {e}')
                write_token = True
        else:
            if age_token > constants_GDrive.TOKEN_MAX_AGE:
                logger.info('token is too old')
            write_token = True
        
        if write_token:
            logger.debug('creating new token file')
            flow = InstalledAppFlow.from_client_secrets_file(secrets, scopes)
            creds = flow.run_local_server(port=0)
    
            try:
                logger.debug(f'writing token file: {token}')
                with open(token, 'w') as tf:
                    tf.write(creds.to_json())
            except OSError as e:
                raise GDriveError(f'error writing token file ({token}): {e}')
        
        return creds
    
    @staticmethod
    def build_service(credentials):
        try:
            service  = build('drive', 'v3', credentials=credentials)
        except google_exceptions.GoogleAuthError as e:
            raise GDriveError(f'error building credentials: {e}')
        return service
    
                
    @property
    def token(self):
        '''token file'''
        return self._token
        
    @token.setter
    def token(self, t_path):
        t_path = Path(t_path)        
        self._token = t_path


    def search(self, name=None, trashed=False, mimeType=None, fuzzy=True, 
               modifiedTime=None, parents=None, dopperator='>',
               fields = [], forcefields=False,
               corpora='user', orderBy='createdTime', driveId='',
               pageSize=constants_GDrive.PAGESIZE, complete=True,
               pageToken=''):
        '''search for objects in google drive by name

        Args:
            name(str): string to search for
            trashed(bool): search in trash when true
            mimeType(str): short mimeType (see MIMETYPES property)
            fuzzy(bool): true: `name contains "value"` false: `name = "value"`
            modifiedTime(str): yyyy-mm-dd string
            dopperator(str): >, < for use with modifiedTime
            parents(str): folder to search within
            fields(list of str): fields to return (see FILE_FIELDS property)
            forcefields(bool): true: use unknown fields, false: reject fields not in FILE_FIELDS
            corpora(str): locations to search (see CORPORA property)
            orderBy(str): order results by (see https://developers.google.com/drive/api/v3/reference/files/list)
            driveId(str): search this shared drive
            pageSize(int): number of results to return per page (default 300)
            complete(bool): true: exhaust all nextPageTokens

        Retruns dict of resutls
            '''
        
        @sleep_and_retry
        @limits(calls=constants_GDrive.CALL_LIMIT, period=constants_GDrive.CALL_PERIOD)
        def _list(pageToken=''):
            logging.debug(f'fettching next page of {pageSize}')            
            try:
                results = self.service.files().list(q=q,
                                                    corpora=corpora,
                                                    includeItemsFromAllDrives=includeItemsFromAllDrives,
                                                    supportsAllDrives=supportsAllDrives,
                                                    fields=fields_string,
                                                    driveId=driveId,
                                                    pageSize=pageSize,
                                                    pageToken=pageToken
                                                    ).execute()
            except HttpError as e:
                raise GDriveError(f'error searching: {type(e)}: {e}')
                
            return results
            
        
        query_build = {
            'name': (name, f'name {"contains" if fuzzy else "="} "{name}"'),
            'trashed': (trashed, f'trashed={trashed}'),
            'mimeType': (mimeType, f'mimeType="{self.MIMETYPES[mimeType] if mimeType in self.MIMETYPES else ""}"'),
            'parents': (parents, f'"{parents}" in parents'),
            'modifiedTime': (modifiedTime, f'modifiedTime{dopperator}"{modifiedTime}"')
        }

        query = [v[1] for k, v in query_build.items() if v[0]]

        if len(fields) < 1:
            fields = self.FIELDS_DEFAULT
        fields = set(fields)

        known_fields = []
        for f in fields:
            if f not in self.FILE_FIELDS:
                if forcefields:
                    logging.warning(f'unknown return field: {f}')
                    known_fields.append(f)
                else:
                    raise GDriveError(f'unknown return field: {f}')
            else:
                known_fields.append(f)

        fields_string = f'nextPageToken, files({",".join(known_fields)})'


        if driveId:
            corpora = 'drive'

        if corpora not in self.CORPORA:
            raise GDriveError(f'unknown `corpora` value: {corpora}')
        else:
            includeItemsFromAllDrives = self.CORPORA[corpora]['params']['includeItemsFromAllDrives']
            supportsAllDrives = self.CORPORA[corpora]['params']['supportsAllDrives']

        q = ' and '.join(query)
        logger.debug(f'QUERY STRING: {q}')
        
        file_list = []
        search_result = _list(pageToken=pageToken)
                    
        token = search_result.get('nextPageToken', False)
        file_list.extend(search_result.get('files', []))
        
        while token and complete:
            logger.debug(f'processing additional pages of results')
            search_result = _list(token)
            token = search_result.get('nextPageToken', False)
            file_list.extend(search_result.get('files', []))


        logging.debug(f'{len(file_list)} total matches returned')
        


        return file_list
        
    def ls(self, *args, **kwargs):
        '''print lis of files in a google drive using any of the search properties'''

        result = self.search(*args, **kwargs)
        for file in result.get('files', []):
            print(('name: {f[name]}, ID:{f[id]}, mimeType:{f[mimeType]}'.format(f=file)))

        return result
        

In [ ]:
sec = './secrets/client_secret_910311278281-bh8qk3kmgk0veri3v8en260e76ipafpj.apps.googleusercontent.com.json'
d = GDrive(secrets=sec, scopes=constants.SCOPES)

In [ ]:
r = d.search(parents='0B9WTleJ1MzaYT2pieWNXYkZtZm8', fields=['parents', 'id', 'name', 'mimeType'], pageSize=100)

In [ ]:
d.CORPORA

In [ ]:
class DC():
    '''dummy class for developing class functions'''
    pass
self = DC()
# self.mimetypes = constants_GDrive.MIMETYPES